In [1]:
import pickle
from src.utils.ml_utils import *
from sklearn.preprocessing import MultiLabelBinarizer

### Loading of the predictive models

In [2]:
# Path to the models
path_genre = 'model_genres.pkl'
path_gender = 'model_Sex.pkl'
path_age = 'model_age_category.pkl'
path_kindness = 'model_kindness.pkl'
path_origin = 'model_Country.pkl'

# Load the models
with open(path_genre, 'rb') as file:
    predict_genres = pickle.load(file)

with open(path_gender, 'rb') as file:
    predict_gender = pickle.load(file)

with open(path_age, 'rb') as file:
    predict_age = pickle.load(file)

with open(path_kindness, 'rb') as file:
    predict_kindness = pickle.load(file)

with open(path_origin, 'rb') as file:
    predict_origin = pickle.load(file)

augmented_alphabet = 'abcdefghijklmnopqrstuvwxyzéèíá'

### Prediction of the movie genre

In [3]:
# Multi-Hot Encoding
genres_list = ['Action & Adventure', 'Drama', 'Comedy', 'Horror & Thriller', 
              'Fantasy & Sci-Fi', 'Historical & War', 'Romance', 'Documentary', 
              'Music & Performance', 'Cult & B-Movies', 'Other']

# Apply MultiLabelBinarizer to encode the genres
mlb = MultiLabelBinarizer(classes=genres_list)
mlb.fit([genres_list])  # Fit the encoder with all possible genres

def feature_creation_g(name):
    df_pred = pd.DataFrame([name], columns=['Name'])
    pred_processor = NameFeatureProcessor('Name', ngram_range=(2,2))
    df_pred = pred_processor.process(df_pred, alphabet=augmented_alphabet, analyze_name=True, diacritic=False, phonetics=False, first_last=True, ngram=False)
    
    # Load pre-trained HashingVectorizer
    with open('hashing_vectorizer.pkl', 'rb') as f:
        vectorizer = pickle.load(f)
    
    # Transform the name using the vectorizer
    ngram_name = vectorizer.transform(df_pred['Name'])
    ngram_name_df = pd.DataFrame(ngram_name.toarray())
    df_pred = pd.concat([df_pred, ngram_name_df], axis=1)
    
    return df_pred

# Function for prediction using the trained model
def predict(df, pred_model):
    df.drop(columns=['Name'], inplace=True)  # Remove 'Name' column
    df.columns = df.columns.astype(str)  # Ensure column names are strings to match model features
    return pred_model.predict(df)

# Function to process the name, extract features, and predict genres
def create_and_predict_genre(name, model):
    df = feature_creation_g(name)  # Feature creation step
    pred = predict(df, model)  # Make prediction with the model
    
    # Decode the binary prediction results to genre names
    decoded_genres = mlb.inverse_transform(pred)
    
    # Display the decoded genre labels
    print(decoded_genres)
    return decoded_genres

### Prediction of the gender/ age/ kindness

In [4]:
def feature_creation(name):
    augmented_alphabet = 'abcdefghijklmnopqrstuvwxyzéèíá'

    df_pred = pd.DataFrame([name], columns=['Name'])
    pred_processor = NameFeatureProcessor('Name',ngram_range=(2,2))
    df_pred =pred_processor.process(df_pred,alphabet = augmented_alphabet, analyze_name = True, diacritic = False, phonetics = False, first_last = True, ngram=False)

    with open(f'hashing_vectorizer.pkl', 'rb') as f:
        vectorizer = pickle.load(f)

    ngram_name = vectorizer.transform(df_pred['Name'])
    ngram_name_df = pd.DataFrame(ngram_name.toarray())
    df_pred = pd.concat([df_pred, ngram_name_df], axis=1)
    return df_pred

def create_and_predict(name, model):
    df = feature_creation(name)  # Feature creation step
    pred = predict(df, model)  # Make prediction with the model

    print(pred)
    return pred

### Prediction of the origin

In [5]:
def feature_creation_o(name):
    augmented_alphabet = 'abcdefghijklmnopqrstuvwxyzéèíá'

    df_pred = pd.DataFrame([name], columns=['Name'])
    pred_processor = NameFeatureProcessor('Name',ngram_range=(2,3))
    df_pred =pred_processor.process(df_pred,alphabet = augmented_alphabet,analyze_name = True, diacritic = False, phonetics = False, first_last = True, ngram=False)

    with open(f'hashing_vectorizer_origin.pkl', 'rb') as f:
        vectorizer = pickle.load(f)

    ngram_name = vectorizer.transform(df_pred['Name'])
    ngram_name_df = pd.DataFrame(ngram_name.toarray())
    df_pred = pd.concat([df_pred, ngram_name_df], axis=1)
    return df_pred

def predict_one_o(df, model):
    df.drop(columns=['Name'],inplace=True)
    df.columns = df.columns.astype(str)
    return model.predict(df)
    
def create_and_predict_origin(name, model):
    df = feature_creation_o(name)
    pred = predict_one_o(df, model)
    print(pred)
    return pred

### Print all the predictions

In [8]:
name_prediction = 'Luca'
genres = create_and_predict_genre(name_prediction, predict_genres)
gender = create_and_predict(name_prediction, predict_gender)
age = create_and_predict(name_prediction, predict_age)
kindness = create_and_predict(name_prediction, predict_kindness)
origin = create_and_predict_origin(name_prediction, predict_origin)

[('Romance', 'Cult & B-Movies')]
['F']
['<17y']
['Bad']
['Romance']
